In [ ]:
import torch
import torchaudio
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [ ]:
transform_train = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize((0.5071, 0.4866, 0.4408), (0.2675, 0.2565, 0.2761)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5071, 0.4866, 0.4408), (0.2675, 0.2565, 0.2761)),
])

In [ ]:
train_data = datasets.CIFAR100(root='./data', train=True, download=True, transform=transform_train)
test_data = datasets.CIFAR100(root='./data', train=False, download=True, transform=transform_test)

train_loader = DataLoader(train_data, batch_size=64, shuffle=True, num_workers=2, pin_memory=True)
test_loader  = DataLoader(test_data,  batch_size=64, shuffle=False, num_workers=2, pin_memory=True)

100%|██████████| 169M/169M [00:14<00:00, 11.8MB/s]


In [ ]:
classes = train_data.classes
print(classes)

['apple', 'aquarium_fish', 'baby', 'bear', 'beaver', 'bed', 'bee', 'beetle', 'bicycle', 'bottle', 'bowl', 'boy', 'bridge', 'bus', 'butterfly', 'camel', 'can', 'castle', 'caterpillar', 'cattle', 'chair', 'chimpanzee', 'clock', 'cloud', 'cockroach', 'couch', 'crab', 'crocodile', 'cup', 'dinosaur', 'dolphin', 'elephant', 'flatfish', 'forest', 'fox', 'girl', 'hamster', 'house', 'kangaroo', 'keyboard', 'lamp', 'lawn_mower', 'leopard', 'lion', 'lizard', 'lobster', 'man', 'maple_tree', 'motorcycle', 'mountain', 'mouse', 'mushroom', 'oak_tree', 'orange', 'orchid', 'otter', 'palm_tree', 'pear', 'pickup_truck', 'pine_tree', 'plain', 'plate', 'poppy', 'porcupine', 'possum', 'rabbit', 'raccoon', 'ray', 'road', 'rocket', 'rose', 'sea', 'seal', 'shark', 'shrew', 'skunk', 'skyscraper', 'snail', 'snake', 'spider', 'squirrel', 'streetcar', 'sunflower', 'sweet_pepper', 'table', 'tank', 'telephone', 'television', 'tiger', 'tractor', 'train', 'trout', 'tulip', 'turtle', 'wardrobe', 'whale', 'willow_tree',

In [ ]:
len(classes)

100

In [ ]:
# Лучшие параметры: {'lr': 0.00021318096016397308, 'batch_size': 32}

In [ ]:
class VGG16_CIFAR(nn.Module):
    def __init__(self, num_classes=100, dropout=0.325):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, 3, padding=1),   nn.BatchNorm2d(64), nn.ReLU(),
            nn.Conv2d(64, 64, 3, padding=1),  nn.BatchNorm2d(64), nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(64, 128, 3, padding=1), nn.BatchNorm2d(128), nn.ReLU(),
            nn.Conv2d(128, 128, 3, padding=1),nn.BatchNorm2d(128), nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(128, 256, 3, padding=1), nn.BatchNorm2d(256), nn.ReLU(),
            nn.Conv2d(256, 256, 3, padding=1), nn.BatchNorm2d(256), nn.ReLU(),
            nn.Conv2d(256, 256, 3, padding=1), nn.BatchNorm2d(256), nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(256, 512, 3, padding=1), nn.BatchNorm2d(512), nn.ReLU(),
            nn.Conv2d(512, 512, 3, padding=1), nn.BatchNorm2d(512), nn.ReLU(),
            nn.Conv2d(512, 512, 3, padding=1), nn.BatchNorm2d(512), nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(512, 512, 3, padding=1), nn.BatchNorm2d(512), nn.ReLU(),
            nn.Conv2d(512, 512, 3, padding=1), nn.BatchNorm2d(512), nn.ReLU(),
            nn.Conv2d(512, 512, 3, padding=1), nn.BatchNorm2d(512), nn.ReLU(),
            nn.MaxPool2d(2),
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(512, 512), nn.ReLU(), nn.Dropout(dropout),
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = VGG16_CIFAR().to(device)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.000117, weight_decay=0.000110)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=100)

In [ ]:
epochs=300
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for i, (images, labels) in enumerate(train_loader):
        images, labels = images.to(device), labels.to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    scheduler.step()
    print(f'Эпоха {epoch+1}/{epochs}, Loss: {running_loss/len(train_loader):.4f}')

Эпоха 1/300, Loss: 3.9234
Эпоха 2/300, Loss: 3.3302
Эпоха 3/300, Loss: 2.8764
Эпоха 4/300, Loss: 2.5645
Эпоха 5/300, Loss: 2.3329
Эпоха 6/300, Loss: 2.1525
Эпоха 7/300, Loss: 2.0014
Эпоха 8/300, Loss: 1.8775
Эпоха 9/300, Loss: 1.7777
Эпоха 10/300, Loss: 1.6701
Эпоха 11/300, Loss: 1.5833
Эпоха 12/300, Loss: 1.5128
Эпоха 13/300, Loss: 1.4417
Эпоха 14/300, Loss: 1.3786
Эпоха 15/300, Loss: 1.3102
Эпоха 16/300, Loss: 1.2521
Эпоха 17/300, Loss: 1.2022
Эпоха 18/300, Loss: 1.1502
Эпоха 19/300, Loss: 1.0985
Эпоха 20/300, Loss: 1.0578
Эпоха 21/300, Loss: 1.0128
Эпоха 22/300, Loss: 0.9683
Эпоха 23/300, Loss: 0.9247
Эпоха 24/300, Loss: 0.8848
Эпоха 25/300, Loss: 0.8469
Эпоха 26/300, Loss: 0.8089
Эпоха 27/300, Loss: 0.7752
Эпоха 28/300, Loss: 0.7409
Эпоха 29/300, Loss: 0.7063
Эпоха 30/300, Loss: 0.6728
Эпоха 31/300, Loss: 0.6364
Эпоха 32/300, Loss: 0.6179
Эпоха 33/300, Loss: 0.5801
Эпоха 34/300, Loss: 0.5548
Эпоха 35/300, Loss: 0.5216
Эпоха 36/300, Loss: 0.5019
Эпоха 37/300, Loss: 0.4737
Эпоха 38/3

In [ ]:
model.eval()
correct = total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy на тесте: {100 * correct / total:.2f}%')

Accuracy на тесте: 70.24%


In [ ]:
torch.save(model.state_dict(), 'cifar100_vgg.pth')

In [ ]:
!pip install optuna -q

import optuna
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split

# Один раз загружаем данные с нормализацией
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize((0.5071, 0.4867, 0.4408), (0.2675, 0.2565, 0.2761))
])

full_train = datasets.CIFAR100(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.CIFAR100(root='./data', train=False, download=True, transform=transform)

# 45k train + 5k val (быстрее чем на всём 50k)
train_dataset, val_dataset = random_split(full_train, [45000, 5000])

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def objective(trial):
    # Параметры для поиска
    lr = trial.suggest_float('lr', 1e-4, 1e-2, log=True)
    batch_size = trial.suggest_categorical('batch_size', [64, 128, 256])
    dropout = trial.suggest_float('dropout', 0.2, 0.6)
    weight_decay = trial.suggest_float('weight_decay', 1e-5, 1e-3, log=True)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)
    val_loader   = DataLoader(val_dataset,   batch_size=256, shuffle=False, num_workers=4, pin_memory=True)

    # Модель (добавил dropout из Optuna)
    model = VGG16_CIFAR(dropout=dropout).to(device)  # используй мою версию VGG из прошлого ответа
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    criterion = nn.CrossEntropyLoss()

    # Быстрое обучение — 10-15 эпох хватает для оценки
    for epoch in range(12):
        model.train()
        for x, y in train_loader:
            x, y = x.to(device), y.to(device)
            pred = model(x)
            loss = criterion(pred, y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

    # Валидация
    model.eval()
    correct = 0
    with torch.no_grad():
        for x, y in val_loader:
            x, y = x.to(device), y.to(device)
            correct += (model(x).argmax(1) == y).sum().item()

    accuracy = correct / len(val_dataset)
    return accuracy

# Запуск
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler(seed=42))
study.optimize(objective, n_trials=30, timeout=None)  # ~3-6 часов на GPU

print("Лучшие параметры:", study.best_params)
print(f"Лучшая валидационная точность: {study.best_value*100:.2f}%")